# TA Scheduling

## Naive Programming

In [3]:
from ortools.sat.python import cp_model

def schedule_tas(availability, ta_requirements, shift_requirements):
    """
    Schedule TAs for shifts based on availability and constraints.
    
    Args:
        availability: A 2D list where availability[i][j] is:
                      - 1: Desired
                      - 0: Undesired
                      - -1: Unavailable
        ta_requirements: A list where ta_requirements[i] is the number of shifts TA i must work.
        shift_requirements: A list where shift_requirements[j] is the number of TAs needed for shift j.

    Returns:
        A dictionary mapping TA indices to their assigned shifts.
    """
    num_tas = len(availability)
    num_shifts = len(availability[0])
    
    # Create the CP-SAT model
    model = cp_model.CpModel()
    
    # Decision variables: x[i][j] is 1 if TA i is assigned to shift j, else 0
    x = {}
    for i in range(num_tas):
        for j in range(num_shifts):
            x[i, j] = model.NewBoolVar(f'x[{i}][{j}]')
    
    # Constraint: TAs must meet their shift requirements
    for i in range(num_tas):
        model.Add(sum(x[i, j] for j in range(num_shifts)) == ta_requirements[i])
    
    # Constraint: Shifts must meet staffing requirements
    for j in range(num_shifts):
        model.Add(sum(x[i, j] for i in range(num_tas)) == shift_requirements[j])
    
    # Constraint: Respect TA availability
    for i in range(num_tas):
        for j in range(num_shifts):
            if availability[i][j] == -1:  # Unavailable
                model.Add(x[i, j] == 0)
    
    # Objective: Maximize TA satisfaction
    objective_terms = []
    for i in range(num_tas):
        for j in range(num_shifts):
            if availability[i][j] == 1:  # Desired
                objective_terms.append(10 * x[i, j])  # High weight for desired
            elif availability[i][j] == 0:  # Undesired
                objective_terms.append(-1 * x[i, j])  # Low weight for undesired
    model.Maximize(sum(objective_terms))
    
    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    # Extract the solution
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        schedule = {}
        for i in range(num_tas):
            schedule[i] = [j for j in range(num_shifts) if solver.Value(x[i, j]) == 1]
        return schedule
    else:
        return None

# Example Input
availability = [
    [1, 0, -1, 1, 1],   # TA 0: Desired, Undesired, Unavailable, Desired, Desired
    [1, 1, 0, -1, 1],   # TA 1: Desired, Desired, Undesired, Unavailable, Desired
    [-1, 1, 1, 0, 1],   # TA 2: Unavailable, Desired, Desired, Undesired, Desired
]
ta_requirements = [2, 3, 2]  # TA 0: 2 shifts, TA 1: 1 shift, TA 2: 2 shifts
shift_requirements = [2, 1, 1, 1, 2]  # Shifts: 2 TAs, 1 TA, 1 TA, 1 TA, 2 TAs

# Solve
solution = schedule_tas(availability, ta_requirements, shift_requirements)
if solution:
    for ta, shifts in solution.items():
        print(f"TA {ta} assigned to shifts: {shifts}")
else:
    print("No feasible solution found.")


TA 0 assigned to shifts: [0, 3]
TA 1 assigned to shifts: [0, 1, 4]
TA 2 assigned to shifts: [2, 4]


In [ ]:
from ortools.sat.python import cp_model

def schedule_tas_debug(availability, ta_requirements, shift_requirements):
    num_tas = len(availability)
    num_shifts = len(availability[0])
    
    print("Availability Matrix:")
    for i in range(num_tas):
        print(f"TA {i}: {availability[i]}")
    
    print("\nTA Requirements:", ta_requirements)
    print("Shift Requirements:", shift_requirements)
    
    # Create the CP-SAT model
    model = cp_model.CpModel()
    
    # Decision variables: x[i][j] is 1 if TA i is assigned to shift j, else 0
    x = {}
    for i in range(num_tas):
        for j in range(num_shifts):
            x[i, j] = model.NewBoolVar(f'x[{i}][{j}]')
    
    # Constraint: TAs must meet their shift requirements
    for i in range(num_tas):
        model.Add(sum(x[i, j] for j in range(num_shifts)) == ta_requirements[i])
        print(f"Added TA {i} shift requirement: {ta_requirements[i]}")
    
    # Constraint: Shifts must meet staffing requirements
    for j in range(num_shifts):
        model.Add(sum(x[i, j] for i in range(num_tas)) == shift_requirements[j])
        print(f"Added shift {j} staffing requirement: {shift_requirements[j]}")
    
    # Constraint: Respect TA availability
    for i in range(num_tas):
        for j in range(num_shifts):
            if availability[i][j] == -1:  # Unavailable
                model.Add(x[i, j] == 0)
                print(f"TA {i} cannot work shift {j}")
    
    # Objective: Maximize TA satisfaction
    objective_terms = []
    for i in range(num_tas):
        for j in range(num_shifts):
            if availability[i][j] == 1:  # Desired
                objective_terms.append(10 * x[i, j])  # High weight for desired
            elif availability[i][j] == 0:  # Undesired
                objective_terms.append(-1 * x[i, j])  # Low weight for undesired
    model.Maximize(sum(objective_terms))
    
    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    # Extract the solution
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        schedule = {}
        for i in range(num_tas):
            schedule[i] = [j for j in range(num_shifts) if solver.Value(x[i, j]) == 1]
        return schedule
    else:
        print("\nNo feasible solution found. Possible reasons:")
        print("- Check if total TA shift requirements match total shift needs.")
        print("- Ensure there are enough available TAs for each shift.")
        print("- Check if constraints are overly strict.")
        return None

# Example Input
availability = [
    [1, 0, -1, 1, 1],   # TA 0: Desired, Undesired, Unavailable, Desired, Desired
    [1, 1, 0, -1, 1],   # TA 1: Desired, Desired, Undesired, Unavailable, Desired
    [-1, 1, 1, 0, 1],   # TA 2: Unavailable, Desired, Desired, Undesired, Desired
]
ta_requirements = [2, 3, 2]  # TA 0: 2 shifts, TA 1: 1 shift, TA 2: 2 shifts
shift_requirements = [2, 1, 1, 1, 2]  # Shifts: 2 TAs, 1 TA, 1 TA, 1 TA, 2 TAs

# Solve
solution = schedule_tas_debug(availability, ta_requirements, shift_requirements)
if solution:
    print("\nFeasible Solution:")
    for ta, shifts in solution.items():
        print(f"TA {ta} assigned to shifts: {shifts}")
else:
    print("No feasible solution found.")


Availability Matrix:
TA 0: [1, 0, -1, 1, 1]
TA 1: [1, 1, 0, -1, 1]
TA 2: [-1, 1, 1, 0, 1]

TA Requirements: [2, 3, 2]
Shift Requirements: [2, 1, 1, 1, 2]
Added TA 0 shift requirement: 2
Added TA 1 shift requirement: 3
Added TA 2 shift requirement: 2
Added shift 0 staffing requirement: 2
Added shift 1 staffing requirement: 1
Added shift 2 staffing requirement: 1
Added shift 3 staffing requirement: 1
Added shift 4 staffing requirement: 2
TA 0 cannot work shift 2
TA 1 cannot work shift 3
TA 2 cannot work shift 0

Feasible Solution:
TA 0 assigned to shifts: [0, 3]
TA 1 assigned to shifts: [0, 1, 4]
TA 2 assigned to shifts: [2, 4]


# General Optimization Practice

## CP-SAT